# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
---
In this project I used OpenCV, NumPy and Jupyter Notebook for road lane lines detection.

---

### ** Here are my steps for completeing this project **


## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline

In [2]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

I put the pipeline into a class. This allows us to have a convinient way to represent developed pipeline as well as add some helpful features such as buffering. Buffering helps us to fill missing frames with averaged approximation and get rid of jittering lines.

In [3]:
from collections import deque # this makes buffer of constant length and automatically keeps it updated
                              # deleting first item in the list

class LaneTracker:
    def __init__(self):
        # for unified size
        self.width           = 960
        self.height          = 540
        # for canny edges
        self.low_threshold   = 50
        self.high_threshold  = 150
        # for blurring/smoothing
        self.kernel_size     = 5
        # for hough lines
        self.rho             = 1
        self.theta           = np.pi/180
        self.threshold       = 40
        self.min_line_len    = 40
        self.max_line_gap    = 250
        # for avoiding outliers
        self.min_left_slope  = 0.5
        self.max_left_slope  = 0.7
        self.min_right_slope = -0.9
        self.max_right_slope = -0.5
        # for filling the gaps
        self.buffer_left          = deque(maxlen=5)
        self.buffer_right         = deque(maxlen=5)
    
    def resize(self, img):
        return cv2.resize(img,(self.width, self.height), interpolation = cv2.INTER_LINEAR)
        
    def yellow_white_masks(self, img):
        # convert to HSL
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        # find white lane
        lower_white = np.array([0, 190, 0], dtype=np.uint8)
        upper_white = np.array([180, 255, 255], dtype=np.uint8)
        white_mask = cv2.inRange(img, lower_white, upper_white)
        # find yellow lane
        lower_yellow = np.array([0, 70, 100], dtype=np.uint8)
        upper_yellow = np.array([100, 250, 255], dtype=np.uint8)
        yellow_mask = cv2.inRange(img, lower_yellow, upper_yellow)
        # combine yellow and white
        yell_white = cv2.bitwise_or(white_mask, yellow_mask)
        masked = cv2.bitwise_and(img, img, mask=yell_white)
        return masked
    
    def grayscale(self, img):
        return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    def gaussian_blur(self, img, kernel_size):
        return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    
    def canny(self, img, low_threshold, high_threshold):
        return cv2.Canny(img, low_threshold, high_threshold)

    def find_vertices(self, imshape):
        bottom_left = (0, imshape[0])
        bottom_right = (imshape[1], imshape[0])
        top_left = ((imshape[1]/2 - imshape[1]*0.03), imshape[0]/1.6)
        top_right = ((imshape[1]/2 + imshape[1]*0.03), imshape[0]/1.6)
        vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
        return vertices
    
    def region_of_interest(self, img):
        vertices  = self.find_vertices(img.shape)
        #defining a blank mask to start with
        mask = np.zeros_like(img)   
        #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
        if len(img.shape) > 2:
            channel_count = img.shape[2]  # i.e. 3 or 4 depending on image
            ignore_mask_color = (255,) * channel_count
        else:
            ignore_mask_color = 255
        #filling pixels inside the polygon defined by vertices with the fill color    
        cv2.fillPoly(mask, vertices, ignore_mask_color)
        #returning the image only where mask pixels are nonzero
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image
    
    def lines_filter(self, lines):
        # Right lines: slope < 0; Left lines:  slope > 0 
        right_line_parameters = []
        left_line_parameters = []
        for line in lines:
            for x1,y1,x2,y2 in line:
                if x1==x2 or y1==y2:
                    continue
                slope  = ((y2-y1)/(x2-x1))
                intercept = y1 - slope*x1
                # initalizing slope control and keep only lines of interest
                if slope > 0 and slope > self.min_left_slope and slope < self.max_left_slope:
                    left_line_parameters.append([slope, intercept])
                if slope < 0 and slope > self.min_right_slope and slope < self.max_right_slope:
                    right_line_parameters.append([slope, intercept])
        # average parameters to form only one line per detected lane
        right_slope_intercept = np.mean(right_line_parameters, axis=0)
        left_slope_intercept = np.mean(left_line_parameters, axis=0)
        return right_slope_intercept, left_slope_intercept

    def line_endpoints(self, shape, slope_intercept):
        # check if some missing values (meaning no line detected)
        try:
            slope = slope_intercept[0]
            intercept = slope_intercept[1]
        except:
            return None
        
        slope = slope_intercept[0]
        intercept = slope_intercept[1]
        y1 = int(shape[0])
        x1 = int((y1 - intercept)/slope)
        y2 = int(shape[0]/1.6)
        x2 = int((y2 - intercept)/slope)
        return [x1, y1, x2, y2]

    def two_hough_lines(self, img, rho, theta, threshold, min_line_len, max_line_gap):
        # hough transform
        lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len,
                                maxLineGap=max_line_gap)
        line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
        # calculate average left and right slope/intercept in order to draw a single line for each lane
        right_slope_intercept, left_slope_intercept = self.lines_filter(lines)
        # find endopints for lines to draw 
        shape = img.shape
        right_line = self.line_endpoints(shape, right_slope_intercept)
        left_line = self.line_endpoints(shape, left_slope_intercept)
        # check if lines detected successfully
        if right_line is None:
            if self.buffer_right:
                right_line = np.mean(self.buffer_right, axis=0, dtype=np.uint32)
        if left_line is None:
            if self.buffer_left:
                left_line = np.mean(self.buffer_left, axis=0, dtype=np.uint32)
        else:
            self.buffer_right.append(right_line)
            self.buffer_left.append(left_line)
            right_line = np.mean(self.buffer_right, axis=0, dtype=np.uint32)
            left_line = np.mean(self.buffer_left, axis=0, dtype=np.uint32)
        # draw lines
        two_lines = [right_line, left_line]
        color = [255, 0, 0]
        thickness = 10
        for line in two_lines:
            cv2.line(line_img, (line[0], line[1]), (line[2], line[3]), color, thickness)
        return line_img

    def weighted_img(self, img, initial_img, α=0.8, β=1., λ=0.):
        return cv2.addWeighted(initial_img, α, img, β, λ)

    def process_image(self, image):
        if image.shape[0] > 540 and image.shape[1] > 960:
            image     = self.resize(image)
        masked    = self.yellow_white_masks(image)
        gray      = self.grayscale(masked)
        blur      = self.gaussian_blur(gray, self.kernel_size)
        edges     = self.canny(blur, self.low_threshold, self.high_threshold)
        roi       = self.region_of_interest(edges)
        two_lines = self.two_hough_lines(roi, self.rho, self.theta, self.threshold, self.min_line_len, 
                                         self.max_line_gap)
        result    = self.weighted_img(two_lines, image)
        return result

# Now let's try our class on videos

In [4]:
tracker_white = LaneTracker()
white_out = 'test_videos_output/solidWhiteRightUpgraded.mp4'
# clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
# white_clip = clip1.fl_image(tracker_white.process_image)
# %time white_clip.write_videofile(white_out, audio=False)

In [5]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_out))

In [6]:
tracker_yellow = LaneTracker()
yellow_out = 'test_videos_output/solidYellowLeftUpgraded.mp4'
# clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4")
# yellow_clip = clip1.fl_image(tracker_yellow.process_image)
# %time yellow_clip.write_videofile(yellow_out, audio=False)

In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_out))

## And, of course, Challenge video


In [8]:
tracker_challenge = LaneTracker()
challenge_out = 'test_videos_output/challengeUpgraded.mp4'
# clip1 = VideoFileClip("test_videos/challenge.mp4")
# challenge_clip = clip1.fl_image(tracker_challenge.process_image)
# %time challenge_clip.write_videofile(challenge_out, audio=False)

In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_out))

### A Fun Part!

Let's drop hardcoded region of interest and see what we've got.

In [10]:
# add some intercept control along with the slope control
class LaneTrackerNoRoi(LaneTracker):
    def __init__(self):
        # for unified size
        self.width           = 960
        self.height          = 540
        # for canny edges
        self.low_threshold   = 50
        self.high_threshold  = 150
        # for blurring/smoothing
        self.kernel_size     = 5
        # for hough lines
        self.rho             = 1
        self.theta           = np.pi/180
        self.threshold       = 40
        self.min_line_len    = 40
        self.max_line_gap    = 250
        # for avoiding outliers
        self.min_left_slope  = 0.5
        self.max_left_slope  = 0.7
        self.min_right_slope = -0.9
        self.max_right_slope = -0.5
        self.min_left_intercept  = 15
        self.max_left_intercept  = 45
        self.min_right_intercept = 630
        self.max_right_intercept = 690
        # for filling gaps
        self.buffer_left          = deque(maxlen=5)
        self.buffer_right         = deque(maxlen=5)
        
    
    def lines_filter(self, lines):
        # Right lines: slope < 0; Left lines:  slope > 0 
        right_line_parameters = []
        left_line_parameters = []
        for line in lines:
            for x1,y1,x2,y2 in line:
                if x1==x2 or y1==y2:
                    continue
                slope  = ((y2-y1)/(x2-x1))
                intercept = y1 - slope*x1
                if slope > 0 and slope > self.min_left_slope and slope < self.max_left_slope\
                and (intercept > self.min_left_intercept and intercept < self.max_left_intercept):
                    left_line_parameters.append([slope, intercept])
                if slope < 0 and slope > self.min_right_slope and slope < self.max_right_slope\
                and (intercept > self.min_right_intercept and intercept < self.max_right_intercept):
                    right_line_parameters.append([slope, intercept])
        right_slope_intercept = np.mean(right_line_parameters, axis=0)
        left_slope_intercept = np.mean(left_line_parameters, axis=0)
        return right_slope_intercept, left_slope_intercept

    def process_image(self, image):
        if image.shape[0] > 540 and image.shape[1] > 960:
            image     = self.resize(image)
        masked    = self.yellow_white_masks(image)
        gray      = self.grayscale(masked)
        blur      = self.gaussian_blur(gray, self.kernel_size)
        edges     = self.canny(blur, self.low_threshold, self.high_threshold)
        two_lines = self.two_hough_lines(edges, self.rho, self.theta, self.threshold, self.min_line_len, 
                                         self.max_line_gap)
        result    = self.weighted_img(two_lines, image)
        return result

In [11]:
tracker_white_noroi = LaneTrackerNoRoi()
white_out_noroi = 'test_videos_output/solidWhiteRightNoRoi.mp4'
# clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
# white_clip_noroi = clip1.fl_image(tracker_white_noroi.process_image)
# %time white_clip_noroi.write_videofile(white_out_noroi, audio=False)

In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_out_noroi))

In [13]:
tracker_challenge_noroi = LaneTrackerNoRoi()
challenge_out_noroi = 'test_videos_output/challengeNoRoi.mp4'
# clip1 = VideoFileClip("test_videos/challenge.mp4")
# challenge_clip_noroi = clip1.fl_image(tracker_challenge_noroi.process_image)
# %time challenge_clip_noroi.write_videofile(challenge_out_noroi, audio=False)

In [14]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_out_noroi))